In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
import joblib
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn import tree

In [2]:
data = pd.read_pickle('train_data.pkl')

In [3]:
data.columns

Index(['DriverNumber', 'Team', 'Driver', 'LapTime_1', 'Sector1Time_1',
       'Sector2Time_1', 'Sector3Time_1', 'LapPercent_1', 'LapPosition_1',
       'Sector1Percent_1', 'Sector1Position_1', 'Sector2Percent_1',
       'Sector2Position_1', 'Sector3Percent_1', 'Sector3Position_1',
       'LapTime_1_1', 'Sector1Time_1_1', 'Sector2Time_1_1', 'Sector3Time_1_1',
       'LapPercent_1_1', 'LapPosition_1_1', 'Sector1Percent_1_1',
       'Sector1Position_1_1', 'Sector2Percent_1_1', 'Sector2Position_1_1',
       'Sector3Percent_1_1', 'Sector3Position_1_1', 'LapTime_2',
       'Sector1Time_2', 'Sector2Time_2', 'Sector3Time_2', 'LapPercent_2',
       'LapPosition_2', 'Sector1Percent_2', 'Sector1Position_2',
       'Sector2Percent_2', 'Sector2Position_2', 'Sector3Percent_2',
       'Sector3Position_2', 'LapTime_1_2', 'Sector1Time_1_2',
       'Sector2Time_1_2', 'Sector3Time_1_2', 'LapPercent_1_2',
       'LapPosition_1_2', 'Sector1Percent_1_2', 'Sector1Position_1_2',
       'Sector2Percent_1_2', '

In [4]:
data = data.dropna()

In [5]:
data.iloc[0]

DriverNumber                           10
Team                           Toro Rosso
Driver                                GAS
LapTime_1          0 days 00:01:46.492000
Sector1Time_1      0 days 00:00:37.686000
                            ...          
PrevPosition1                        17.0
PrevLapPercent2                  0.015587
PrevPosition2                         6.0
PrevLapPercent3                  0.050897
PrevPosition3                        20.0
Name: 60, Length: 66, dtype: object

In [6]:
train_data = data[data['EventDate'] <= np.datetime64('2022-06-30')]
test_data = data[data['EventDate'] > np.datetime64('2022-06-30')]

In [7]:
train_data = train_data.select_dtypes(include=np.float64)

In [8]:
for col in train_data.columns:
    percentiles = train_data[col].quantile([0, 0.99]).values
    train_data.loc[train_data[col] >= percentiles[1], col] = percentiles[1]

In [9]:
# norm = Normalizer()

In [10]:
# scores = {}
# for col in train_data.select_dtypes(include=np.float64).columns:
#     scores[col] = r2_score(train_data['Position'], train_data[col])

In [33]:
X_cols =['LapPercent_1', 'LapPercent_2', 'PrevPosition1',
       'PrevPosition2', 'PrevPosition3', 'PrevLapPercent1',
       'PrevLapPercent2', 'PrevLapPercent3']

In [34]:
train_X = train_data[X_cols]
train_y = train_data['LapPercent']

In [35]:
train_X.to_pickle('train_X.pkl')

In [36]:
train_X_norm = (train_X - train_X.mean()) / train_X.std()
# train_y = (train_y - train_y.mean()) / train_y.std()

In [37]:

# for col in train_X.columns:
#     lower_percentile = 0.05
#     higher_percentile = 0.95

#     low, high = train_X[col].quantile([lower_percentile, higher_percentile])

#     #cap values above high to high
#     train_X.loc[:,col] = train_X.loc[:,col].apply(lambda x: high if x > high else x)#[train_X[col] > high] = high


In [38]:
# train_X = (train_X - train_X.mean()) / train_X.std()

In [39]:
train_X_norm.columns.size

8

In [40]:
train_X_norm

,LapPercent_1,LapPercent_2,PrevPosition1,PrevPosition2,PrevPosition3,PrevLapPercent1,PrevLapPercent2,PrevLapPercent3
60,0.218996,0.244418,1.241772,-0.694936,1.782656,0.986220,-0.258550,2.246792
61,-0.844265,-0.273708,-0.348679,0.374852,0.540552,-0.071410,0.425786,1.112222
62,0.198736,-0.675483,0.534905,0.553151,0.185666,0.301871,0.472125,0.753381
63,0.506384,0.451668,1.595205,1.622939,1.427769,1.264212,1.471736,1.667193
64,0.292531,0.014443,1.418489,1.801237,0.717996,1.131122,1.540418,1.310112
...,...,...,...,...,...,...,...,...
1694,0.679789,0.252766,1.771922,0.909747,0.008222,1.744452,0.345605,0.080201
1695,-0.550352,-1.101052,-0.171962,-0.160042,1.072882,-0.155193,-0.131630,0.611394
1696,-1.213296,-1.214500,-1.055546,-1.408129,-1.233881,-1.059010,-1.163601,-1.009378
1697,0.698523,1.504480,1.418489,1.622939,1.782656,1.455683,1.693694,1.482513


In [41]:
model = linear_model.LinearRegression()
model.fit(train_X_norm, train_y)

LinearRegression()

In [42]:
# params = {}
# best_model = GridSearchCV(model, param_grid = params).fit(train_X, train_y)

In [43]:
pd.concat([pd.DataFrame(train_X_norm.columns),pd.DataFrame(np.transpose(model.coef_))], axis = 1)


,0,0
0,LapPercent_1,0.002037
1,LapPercent_2,0.004888
2,PrevPosition1,0.000948
3,PrevPosition2,0.000976
4,PrevPosition3,0.003447
5,PrevLapPercent1,0.000867
6,PrevLapPercent2,0.000393
7,PrevLapPercent3,-0.000291


In [44]:
test_X = test_data[X_cols]
test_y = test_data['LapPercent']

In [45]:
test_X = (test_X - train_X.mean()) / train_X.std()

In [46]:
pred = model.predict(test_X)

In [47]:
r2_score(train_data['LapPercent'], train_data['PrevPosition1'])

-393744.2547375366

In [48]:
results = pd.DataFrame({'pred': pred, 'actual': test_y})

In [49]:
results

,pred,actual
1704,0.016232,0.007233
1705,0.047016,0.043388
1706,0.029611,0.030566
1707,0.041610,0.027883
1709,0.030003,0.016766
...,...,...
2049,0.034247,0.058647
2050,0.040096,0.020070
2052,0.018123,0.014366
2053,0.015492,0.039327


In [50]:
results.describe()

,pred,actual
count,300.000000,300.000000
mean,0.020329,0.018256
std,0.010379,0.028250
min,0.000135,0.000000
25%,0.011793,0.007552
50%,0.020891,0.015483
75%,0.027268,0.023614
max,0.047016,0.455956


In [51]:
mean_squared_error(results['pred'], results['actual'])

0.0007133550172953425

In [52]:
joblib.dump(model, 'model.pkl')

['model.pkl']